In [ ]:
%cd ~
import matplotlib
matplotlib.use("PDF")
import pyfits,glob,time,scipy, sys
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell, minimize
from scipy.special import airy,jv

topdir='/Users/cslage/Research/LSST/code/GUI/'
#thedir=topdir+'brighter_fatter/20170831_002_spots/'
thedir=topdir+'brighter_fatter/20170609-bf-30um/'
sys.path.append('/Users/cslage/Research/LSST/code/galsim-newgit/GalSim/devel/brighter-fatter/forward_model_varying_i')

import  forward
sys.path.append('/Users/cslage/Learning/code/languages/python/extensions/forward_model_convolved')
#import  forward_convolved

%cd $thedir


configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'



In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'ITL-3800C-002_spot_spot_810_20170609??????.fits'))
print len(zfilelist)

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in [13]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 50
    xcoomax = 450
    ycoomin = 1400
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff#xcoord - xint - 1.0
            spotlist.yoffset[n] = yoff#ycoord - yint - 1.0     
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
for segmentnumber in range(1,17):
    hdr=pyfits.getheader(zfilelist[0],segmentnumber)
    extname = hdr['EXTNAME']
    print "Segment # %d is %s"%(segmentnumber,extname)


In [ ]:
nx = 11
ny = 11
numspots =400
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

print "nstamps = %d"%spotlist.nstamps
starttime = time.time()
param0 = [1.00, 1.00]

args = ()#(spotlist)
Result = fmin_powell(FOM, param0, args)
elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed
testn = 0
spot = 228
sigmax = abs(Result[0])
sigmay = abs(Result[1])
ADU_correction = Area(-0.5,0.5,-0.5,0.5,sigmax, sigmay,1.0)
Imax = spotlist.imax[spot] 
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax) 

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8,10])
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8,10])
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_20170609_02Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:
from scipy.ndimage import shift
def DrizzleSpotList(spotlist, nzoom):
    stampstack=zeros((spotlist.nx*nzoom,spotlist.ny*nzoom))
    regridstamp=zeros((spotlist.nx*nzoom,spotlist.ny*nzoom))
    for k in range(spotlist.nstamps):
        xshift,yshift=round(spotlist.xoffset[k]*nzoom),round(spotlist.yoffset[k]*nzoom)
        for i in range(spotlist.nx):
            for j in range(spotlist.ny):
                i0,i1=nzoom*i,nzoom*(i+1)
                j0,j1=nzoom*j,nzoom*(j+1)
                regridstamp[i0:i1,j0:j1]=spotlist.data[i,j,k]
        shiftstamp=shift(regridstamp,(-xshift,-yshift),order=0)
        stampstack+=shiftstamp
    stampstack=stampstack/spotlist.nstamps
    return stampstack

def moments(data, center):
    y, x = np.indices((data.shape))
    x = (x - center[0] + 0.5) / nzoom
    y = (y - center[1] + 0.5) / nzoom
    mx = (data * x).sum() / data.sum()
    my = (data * y).sum() / data.sum()
    mx2 = (data * (x-mx) * (x-mx)).sum() / data.sum()
    my2 = (data * (y-my) * (y-my)).sum() / data.sum()
    mxy = (data * (x-mx) * (y-my)).sum() / data.sum()
    return [mx, my, mx2, my2, mxy]



In [ ]:
nzoom = 10
halfstampsize = (nx + 1) / 2
stampstack = DrizzleSpotList(spotlist, nzoom)
figure()
imshow(stampstack,cmap='rainbow',aspect='equal',origin='lower', interpolation='nearest'),colorbar()
show()
#savefig("Drizzle_Test")
close()

#[mx, my, mx2, my2, mxy] = moments(stampstack, (45,45))
#print mx, my, mx2, my2, mxy

In [ ]:
def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    rx=linspace(0,halfstampsize*np.sqrt(2),round(halfstampsize*nzoom*np.sqrt(2)))
    return rx,radialprofile 

figure(figsize=(10,5))
subplot(121)
imshow(stampstack,cmap='rainbow',aspect='equal',origin='lower'),colorbar()

for i in range(1,2*halfstampsize):
    axvline(i*nzoom,color='w')
    axhline(i*nzoom,color='w')
axis('off')
subplot(122)
rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))

plot(rx,rp/rp.max())
#yscale('log')

foox=np.linspace(0,8,100)
foosig=1.35
fooy=np.exp(-foox**2/(2.0*foosig**2))
plot(foox,fooy)

show()
close()

print rp.max()

In [ ]:
figure(figsize=(12,5))
rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
plot(rx,rp/rp.max(),label='Data stacked')
#yscale('log')

foox=np.linspace(.001,8,100)
foosig=1.30
fooy=np.exp(-foox**2/(2.0*foosig**2))
plot(foox,fooy,label='Gaussian')
fooy=(jv(1,foox/1.0)/foox)**2
plot(foox,fooy/fooy[0],label='Airy')
legend(loc='upper right')
axis([0,8,1e-4,1])
show()
close()

In [ ]:
def ConvolvedGaussianFull(x, y, sigmax=1.0, sigmay=1.0, c=0.5):
    ssigx2 = sigmax * sigmax * 2.0
    ssigy2 = sigmay * sigmay * 2.0
    nx = 100
    ny = 100
    xmax = 10.0
    ymax = 10.0
    data = Array2d(-xmax,xmax,nx,-ymax,ymax,ny)
    sum = 0.0
    c2 = c * c
    for i in range(nx):
        x1 = data.x[i]
        for j in range(ny):
            y1 = data.y[j]
            r2 = (x-x1)*(x-x1) + (y-y1)*(y-y1)
            if r2 < c2:
                sum += exp(-x1*x1/ssigx2) * exp(-y1*y1/ssigy2) * data.dx * data.dy
    return sum 

def Gaussian(x, y, sigmax=1.0, sigmay=1.0):
    ssigx = sigmax * sqrt(2.0)
    ssigy = sigmay * sqrt(2.0)
    sr = sqrt((x/ssigx)**2 + (y/ssigy)**2)
    return exp(-sr**2)

def CGFOM(params):
    fom = 0.0
    [sigma, ctest] = params
   
    for i,x in enumerate(rx):
        cg = ConvolvedGaussianFull(x,0,sigmax=sigma, sigmay=sigma, c=ctest)
        cgnorm = ConvolvedGaussianFull(0,0,sigmax=sigma, sigmay=sigma, c=ctest)
        #print sigma, ctest, cgnorm
        #print x, cg, rp[i]/rp.max()
        fom += square(cg/cgnorm-rp[i]/rp.max())

    #print sigma, ctest, fom
    return fom

def GFOM(params):
    fom = 0.0
    [sigma] = params
   
    for i,x in enumerate(rx):
        cg = Gaussian(x,0,sigmax=sigma, sigmay=sigma)
        #print x, cg, rp[i]/rp.max()
        fom += square(cg-rp[i]/rp.max())

    #print sigma, ctest, fom
    return fom

close()
rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
figure(figsize=(12,5))
subplots_adjust(hspace=0.5)
subplot(2,1,1)
title("Spot Profile")
plot(rx,rp/rp.max(),label='Data stacked')

param0 = [1.2, 0.9]
args = ()#(spotlist)
CGResult = fmin_powell(CGFOM, param0, args)
print CGResult
NormFactor = ConvolvedGaussianFull(0,0,sigmax=CGResult[0], sigmay=CGResult[0], c=CGResult[1])

CGfooy = []
for x in rx:
    CGfooy.append(ConvolvedGaussianFull(x,0,sigmax=CGResult[0], sigmay=CGResult[0], c=CGResult[1]) / NormFactor)
CGfooy=array(CGfooy)
plot(rx,CGfooy,label='Convolved Gaussian')

param0 = [1.0]
args = ()#(spotlist)
GResult = fmin_powell(GFOM, param0, args)
print GResult
Gfooy=Gaussian(rx,0,sigmax=GResult, sigmay=GResult)
plot(rx,Gfooy,label='Gaussian')

#yscale('log')


legend(loc='upper right')
axis([0,8,1e-4,1])
ylabel("Profile(Peak=1.0)")
xlabel("R (Pixels)")

subplot(2,1,2)
title("Error - percent full scale")
plot(rx,rp/rp.max(),label='Data stacked')

error = abs(CGfooy-rp/rp.max())*100.0
print error.max()
plot(rx,error,label='Error Convolved Gaussian')

error = abs(Gfooy-rp/rp.max())*100.0
plot(rx,error,label='Error Gaussian')

#yscale('log')


legend(loc='upper right')
axis([0,8,1e-4,1])
ylim(0,2)
ylabel("Percent Error")
xlabel("R (Pixels)")
savefig("Drizzled_Profile_Error_07Nov17.pdf")
show()
close()

In [ ]:

class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])

def CGIntegrand(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xh
    ym = yc - xh
    f = yp * erf(yp / ssx) - ym * erf(ym / ssx)
    f += ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) / sqrt(pi)
    f *= erf((y+yh) / ssy)
    return f

def ICGStep(xh, yh, sigmax, sigmay, c):
    ny = 100
    dy = 2 * c / (ny - 1)
    sum = 0.0
    y = -c + dy / 2.0
    for i in range(ny-1):
        sum += dy * CGIntegrand(y,xh,yh,sigmax,sigmay,c)
        y += dy
    return sum

def ICGFull(xl, xh, yl, yh, sigmax, sigmay, c):
    sum = ICGStep(xh,yh,sigmax,sigmay,c) + ICGStep(xl,yl,sigmax,sigmay,c)
    sum -= ICGStep(xh,yl,sigmax,sigmay,c) + ICGStep(xl,yh,sigmax,sigmay,c)
    return sum
        
def CGFOM(params):
    [sigmax, sigmay] = params
    #[sigmax, sigmay, c] = params
    c=1.63
    result = forward_convolved.forward_convolved(spotlist,sigmax,sigmay,c)
    return result


nx = 11
ny = 11
numspots =400
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

print "nstamps = %d"%spotlist.nstamps
starttime = time.time()
#param0 = [1.00, 1.00, 0.7]
param0 = [1.00, 1.00]

args = ()#(spotlist)
Result = fmin_powell(CGFOM, param0, args)

elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed


In [ ]:
testn = 0
spot = 228
sigmax = abs(Result[0])
sigmay = abs(Result[1])
c = Result[2]
norm = ICGFull(-10.0,10.0,-10.0,10.0,sigmax,sigmay,c)
ADU_correction = ICGFull(-0.5,0.5,-0.5,0.5,sigmax, sigmay,c) / norm
Imax = spotlist.imax[spot] 
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Imax * ICGFull(xl, xh, yl, yh, sigmax, sigmay, c) / norm

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,45000)
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,45000)
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_Convolved_07Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:
def Yfunc(y):
    x=0.0
    return exp(-y*y) * erf(x-sqrt(1-y*y))

x = linspace(-20.0,20.0,500)
y = Yfunc(x)
plot(x,y)
show()

In [ ]:
def WedgeArea(x,y0,c):
    y = sqrt(c**2-(sqrt(2*c*y0 - y0**2)-x)**2) - c + y0
    z = sqrt(x**2 + y**2)
    theta = arcsin(z / (2*c))
    A = x*y/2 + c**2 * theta - z/2 * sqrt(c**2 - z**2/4.0)
    return A

In [ ]:
y0=0.2
c=1.0
x0=(sqrt(c**2 - (c-y0)**2))
print "xmax = %f"%x0
print WedgeArea(.3,y0,c)

print WedgeArea(0,y0,c)
x=linspace(0,x0,100)
y=WedgeArea(x,y0,c)
plot(x,y)
show()
close()

In [ ]:
def CGIntegrand(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xh
    ym = yc - xh
    f = yp * erf(yp / ssx) - ym * erf(ym / ssx)
    f += ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) / sqrt(pi)
    f *= erf((y+yh) / ssy)
    return f


print CGIntegrand(0,-1.0,-1.0,1.0,2.0,1.5)